In [2]:
# load data
dt1 <- read.csv("../output_combined/a0_combinedMonthly.csv", sep = ",")
summary(dt1)

      X                 X.AEX          ASML.AS             AIR        
 Length:1164        Min.   :179.7   Min.   :  1.932   Min.   : 3.532  
 Class :character   1st Qu.:338.8   1st Qu.: 18.709   1st Qu.:13.685  
 Mode  :character   Median :453.9   Median : 42.917   Median :20.619  
                    Mean   :469.4   Mean   :143.424   Mean   :24.042  
                    3rd Qu.:558.2   3rd Qu.:144.914   3rd Qu.:30.793  
                    Max.   :922.7   Max.   :950.154   Max.   :71.078  
                    NA's   :805     NA's   :847       NA's   :805     
      ING             WKL.AS             STLA            AD.AS       
 Min.   : 2.826   Min.   :  5.588   Min.   : 1.716   Min.   : 4.868  
 1st Qu.: 6.983   1st Qu.: 10.359   1st Qu.: 3.784   1st Qu.: 6.717  
 Median : 9.158   Median : 13.556   Median : 6.432   Median :14.288  
 Mean   :10.200   Mean   : 30.451   Mean   : 8.342   Mean   :14.852  
 3rd Qu.:12.172   3rd Qu.: 33.542   3rd Qu.:12.386   3rd Qu.:20.408  
 Max.   :24.

In [ ]:
allColumns <- colnames(dt1)
# remove Perioden
allColumns_1 <- allColumns[c(-1)]

In [ ]:
##########################
# FOR LOOP
##########################

[1] "X"                                                                                
  [2] "X.AEX"                                                                            
  [3] "ASML.AS"                                                                          
  [4] "AIR"                                                                              
  [5] "ING"                                                                              
  [6] "WKL.AS"                                                                           
  [7] "STLA"                                                                             
  [8] "AD.AS"                                                                            
  [9] "PHG"                                                                              
 [10] "X.FVX"                                                                            
 [11] "X.TNX"                                                                            
 [12] "X.TYX"                                                                            
 [13] "Bankruptcies"                                                                     
 [14] "BusinessOutlook_Industry"                                                         
 [15] "BusinessOutlook_Retail"                                                           
 [16] "Consumentenvertrouwen_1"                                                          
 [17] "EconomischKlimaat_2"                                                              
 [18] "Koopbereidheid_3"                                                                 
 [19] "EconomischeSituatieLaatste12Maanden_4"                                            
 [20] "EconomischeSituatieKomende12Maanden_5"                                            
 [21] "FinancieleSituatieLaatste12Maanden_6"                                             
 [22] "FinancieleSituatieKomende12Maanden_7"                                             
 [23] "GunstigeTijdVoorGroteAankopen_8"                                                  
 [24] "CPI_1"                                                                            
 [25] "CPIAfgeleid_2"                                                                    
 [26] "MaandmutatieCPI_3"                                                                
 [27] "MaandmutatieCPIAfgeleid_4"                                                        
 [28] "ProducerConfidence_1"                                                             
 [29] "ExpectedActivity_2"                                                               
 [30] "CrudeOil_x"                                                                       
 [31] "Gold_x"                                                                           
 [32] "X.GSPC_x"                                                                         
 [33] "BTC.USD_x"                                                                        
 [34] "Crude.oil..average"                                                               
 [35] "Crude.oil..Brent"                                                                 
 [36] "Crude.oil..Dubai"                                                                 
 [37] "Crude.oil..WTI"                                                                   
 [38] "Coal..Australian"                                                                 
 [39] "Coal..South.African..."                                                           
 [40] "Natural.gas..US"                                                                  
 [41] "Natural.gas..Europe"                                                              
 [42] "Liquefied.natural.gas..Japan"                                                     
 [43] "Natural.gas.index"                                                                
 [44] "Cocoa"                                                                            
 [45] "Coffee..Arabica"                   

In [5]:
for(colName in allColumns_1){ 

#colName <- "Bankruptcies"
print(colName)

# connects all the data
Key1 <- paste(Sys.Date(), "_", colName, sep="")

series1 <- ts(dt1[colName], frequency = 4, start=c(1995,1))

# if all values negative, go to next value
if (sum(is.na(series1)) > 50) {next}

# drop nas, date will remain the same
series1 <- na.omit(series1, cex.main = 6, col.main = "darkgreen")

plot(series1, main = colName)

# check if series contains negative numbers
if (any(as.numeric(series1) < 0)) {
  modelform <- "AAA"
  print("Data contains negative numbers")

} else {

  #########################
  # Which model to use
  #########################

  ###########################
  # Trend or not?
  ###########################

  p_value <- try(notrend_test(series1)$p.value)
  print(p_value)

  if (is.numeric(p_value) == FALSE) {
    print("assume: Has Trend")
    trendtype <- "A"
  } else {
      if (p_value < 0.05) { ############################# CHECK THIS ######################
        print("Has Trend")
        trendtype <- "A"
      } else {
        print("No Trend")
        trendtype <- "N"
        }
  }

  print(trendtype)

  ###########################
  # Additive or multiplictive?
  ###########################

  decompose_series1 <- decompose(series1, "multiplicative")
  decompose_series1_multiplicative <- decompose_series1$random
  muladd_mul <- sqrt(mean(abs(decompose_series1_multiplicative)^2, na.rm=TRUE))

  decompose_series1 <- decompose(series1, "additive")
  decompose_series1_additive <- decompose_series1$random
  muladd_add <- sqrt(mean(abs(decompose_series1_additive)^2, na.rm=TRUE))

  if (muladd_mul < muladd_add) {
    print("Use Multiplicative")
    errortype <- "M"
    } else {
      print("Use Additive")
      errortype <- "A"
  }

  print(errortype)

  ###########################
  # Seasonnal or not
  ###########################

  season_Check <- isSeasonal(series1)

  if (season_Check == TRUE) {
    print("Use Seasonal")
    seasontype <- "A"
    } else {
      print ("Use Non-Seasonal")
      seasontype <- "N"
    }

  print(seasontype)

  modelform <- str_c(c(errortype, trendtype, seasontype), collapse = "")
  } # end if for negative numbers

###############
# above, if negative numbers use simple method
###############

fit <- ets(series1, model=modelform, damped=FALSE)

############
# test period
#############
test_period <- 4

h1 <- test_period
train <- head(series1, round(length(series1) - h1))
test <- tail(series1, h1)

fit <- ets(train, model=modelform, damped=FALSE)
forecasted1 <- forecast(fit, h=h1)

png(filename=paste("output/figures/", Key1, "TrainTestForecast.png", sep = "_"))
print(autoplot(forecasted1, series = "In-sample Forecast", include=h1+16) + autolayer(test, series = "Historical Data") + ggtitle(colName)) + ylab("Historical + Forecast")
dev.off()

####################
# final forecast
####################
fit <- ets(series1, model=modelform, damped=FALSE)
forecast_oneMonth <- forecast(fit, h=1)

png(filename=paste("output/figures/", Key1, "final_forecasts.png", sep = "_"))
print(autoplot(tail(series1, 20)) + autolayer(forecast_oneMonth) + ggtitle(colName))
dev.off()

################################
# Saving
################################

###
# Raw Data
###

data <- data.frame(
  SeriesName   = colName, 
  DateAnalysis = Sys.Date(), 
  ETSmodel = modelform,
  ObservationDate = as.yearqtr(time(series1)),  ##### Careful here, yearmonth is not the same as yearqtr
  RawData = series1
)
data$Key1 <- Key1

colnames(data) <- c("SeriesName", "DateAnalysis", "ETSmodel", "ObservationDate", "RawData", "Key1")
write.table(data, file = paste("output/forecasts/", Key1, "RawData.csv", sep="_"), sep =",",row.names = FALSE)

###
# TrainTestForecast
###
forecast_tibble <- as.data.frame(forecasted1)
forecast_tibble$Key1 <- Key1 

write.table(forecast_tibble, file = paste("output/forecasts/", Key1, "TrainTestForecast.csv", sep="_"), sep =",",row.names = FALSE)

###
# finalForecast
###
forecast_oneMonth <- forecast(fit, h=1)
finalForecast <- as.data.frame(forecast_oneMonth, row.names = NULL)
finalForecast$Key1 <- Key1
finalForecast <- tibble::rownames_to_column(finalForecast, "Forecast_Period")  

write.table(finalForecast, file = paste("output/forecasts/", Key1, "final_forecasts.csv", sep="_"), sep =",",row.names = FALSE)

} 
############# END LOOP ##############
############# END LOOP ##############
############# END LOOP ##############

ERROR: Error: object 'allColumns_1' not found
